In [1]:
# Used for selecting the 6 best features per cluster. 
# We're using mean squared error of each variable vs. the ALSFRS_score, and take the best 6. 

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model

In [3]:
#proact_data = pd.read_csv('../train_data.csv', sep = '|', index_col=False)
vectorized_data = pd.read_csv('../train_data_vectorized.csv', sep='|', index_col=0)
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col=0)
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col=0)
feature_groups = pickle.load( open('../feature_groups.pickle', 'rb') )

X = clusters.join(vectorized_data)
Y = clusters.join(slope)
X.head()


,cluster,F,M,Asian,Black,Hispanic,Other,Unknown,White,family_ALS_hist_last,...,ALSFRS_Total_median_zscore,ALSFRS_Total_std_zscore,ALSFRS_Total_pct_diff_zscore,ALSFRS_Total_last_zscore,ALSFRS_Total_mean_slope_zscore,temperature_mean_zscore,temperature_median_zscore,temperature_std_zscore,temperature_pct_diff_zscore,temperature_mean_slope_zscore
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,1,0,0,0,0,0,0,1,0.208791,...,-0.256048,0.238113,0.085062,-0.118584,-1.485349,-2.030917e+00,-3.390882e+00,-2.608928e-01,-0.217011,0.284236
649,1,1,0,0,0,0,0,0,1,0.208791,...,1.037869,-0.702395,0.236862,1.079684,0.011502,-1.903188e-14,3.568549e-14,-8.058387e-17,0.000000,0.000000
1234,2,0,1,0,0,0,0,0,1,0.208791,...,-1.734810,0.766708,0.236862,-1.488034,-0.264381,-1.903188e-14,3.568549e-14,-8.058387e-17,0.000000,0.000000
2492,1,0,1,0,0,0,0,0,1,0.208791,...,-0.071202,-1.273871,0.236862,0.052597,0.215110,5.806198e-01,1.129175e+00,1.198237e-02,-0.757395,-0.657605
2956,0,1,0,0,0,0,0,0,1,0.208791,...,0.298488,-0.130919,0.236862,0.394959,-0.250279,-1.903188e-14,3.568549e-14,-8.058387e-17,0.000000,0.000000


In [9]:
#from vectorizer_beta import * 
best_features_per_cluster = {}

for c in clusters['cluster'].unique():
    seg_X, seg_Y = X[X['cluster'] == c], Y[Y['cluster'] == c]
    seg_Y = seg_Y.fillna(seg_Y.mean())
    
    score_per_feature = {}
    
    for feature_group, feature_names in feature_groups.iteritems():
        regr = linear_model.LinearRegression()
        X_feature_fam = seg_X[list(feature_names)]
        regr.fit(X_feature_fam, seg_Y)
        score_per_feature[feature_group] = regr.score(X_feature_fam, seg_Y)
    
    best_features_per_cluster[c] = sorted(sorted(score_per_feature, key=score_per_feature.get)[:6])
    
best_features_per_cluster

{0: ['Age', 'BMI', 'Gender', 'Race', 'family_ALS_hist', 'height'],
 1: ['Age', 'BMI', 'Gender', 'fvc_percent', 'height', 'respiratory_rate'],
 2: ['Age', 'BMI', 'Gender', 'Race', 'family_ALS_hist', 'height']}

In [10]:
import pickle 
with open("../best_features_per_cluster.pickle", "wb") as output_file:
    pickle.dump(best_features_per_cluster, output_file)
